# Data exploration
This notebook contains some tests about data import and some data viz tools

In [1]:
DATA_FOLDER = './data/'
MOVIES_FOLDER = DATA_FOLDER + 'movies_summaries/'
PLOT_SUMMARY_FOLDER = DATA_FOLDER + 'corenlp_plot_summaries'

REPORT_FOLDER = './gen/reports/'
ETHNICITY_FILE = './gen/ethnicities.tsv'

CHARACTERS_FILE = MOVIES_FOLDER + 'character.metadata.tsv'
MOVIES_FILE = MOVIES_FOLDER + 'movie.metadata.tsv'
PLOT_SUMMARIES_FILE = MOVIES_FOLDER + 'plot_summaries.txt'
TROPES_FILE = MOVIES_FOLDER + 'tvtropes.clusters.txt'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import re

from pandas_profiling import ProfileReport
%matplotlib inline

# Movies

In [30]:
movies_columns = ['wiki_movie_id', 'freebase_movie_id', 'name', 'release_date', 'box_office_revenue', 'runtime', 'languages', 'countries', 'genres']
movies = pd.read_csv(MOVIES_FILE, sep='\t', names=movies_columns)

print(movies.shape)
movies.head()

(81741, 9)


,wiki_movie_id,freebase_movie_id,name,release_date,box_office_revenue,runtime,languages,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [34]:
def replace_unknown(df, label):
    return df[label].replace("{}", "{\"\": \"Unknown\"}")

movies1 = movies.copy()

movies1['countries'] = replace_unknown(movies1, 'countries')
movies1['genres'] = replace_unknown(movies1, 'genres')
movies1['languages'] = replace_unknown(movies1, 'languages')

movies1.loc[200]

wiki_movie_id                            30693170
freebase_movie_id                      /m/0g9z2v0
name                                       Harumi
release_date                                  NaN
box_office_revenue                            NaN
runtime                                       NaN
languages                         {"": "Unknown"}
countries                         {"": "Unknown"}
genres                {"/m/0jtdp": "Documentary"}
Name: 200, dtype: object

In [37]:
def extract_feature(json):
    if json is np.nan:
        return np.nan
    return list(ast.literal_eval(json).values())

movies2 = movies1.copy()

movies2["countries"] = movies2["countries"].apply(extract_feature)
movies2["genres"] = movies2["genres"].apply(extract_feature)
movies2["languages"] = movies2["languages"].apply(extract_feature)
movies2["countries"].apply(len).value_counts()

1     72523
2      6718
3      1747
4       539
5       131
6        47
7        17
8         9
9         4
11        2
14        2
10        1
13        1
Name: countries, dtype: int64

In [38]:
def filter_with_countries(df, target_countries, mode):
    if mode == 'all':
        return df.where(df["countries"].apply(lambda x: all(country in x for country in target_countries))).dropna()
    elif mode == 'any':
        return df.where(df["countries"].apply(lambda x: any(country in x for country in target_countries))).dropna()
    elif mode == 'only':
        return df.where(df["countries"].apply(lambda x: set(x) == set(target_countries))).dropna()
    else:
        raise ValueError('mode must be one of [all, any, only]')

target_countries = ['Serbia']
filter_with_countries(movies2, target_countries, mode='any')

,wiki_movie_id,freebase_movie_id,name,release_date,box_office_revenue,runtime,languages,countries,genres
5990,189776.0,/m/019z8l,Underground,1995-04-01,171082.0,170.0,"[French Language, Serbo-Croatian, German Langu...","[Serbia and Montenegro, Bulgaria, France, Germ...","[World cinema, Drama, Comedy, War film, Black ..."
6055,17366149.0,/m/043qtbn,St. George Shoots the Dragon,2008-10,325265.0,120.0,"[Serbo-Croatian, Serbian language]","[Bulgaria, Serbia, Bosnia and Herzegovina]","[History, Drama, World cinema, War film]"
16155,21133929.0,/m/05c1msq,Here and There,2009-03-01,18000.0,81.0,"[English Language, Serbian language]","[United States of America, Serbia, Germany]","[Comedy film, Drama, Indie, World cinema]"
45046,14882264.0,/m/03h0b44,Charleston & Vendetta,2008-01-30,634620.0,86.0,[Serbian language],[Serbia],"[Fantasy, Drama, Comedy]"
72338,30555534.0,/m/0j50wym,Montevideo: Taste of a Dream,2010-12-20,2192931.0,140.0,"[Bulgarian Language, Serbian language]",[Serbia],"[Adventure, History, Comedy film, Sports, Comi..."


# Characters

In [24]:
character_columns = ['wiki_movie_id', 'freebase_movie_id', 'm_release_date', 'name', 'a_dob', 'a_gender', 'a_height', 'a_ethnicity_freebase_id', 'a_name', 'a_age_at_release', 'freebase_char/a_map', 'freebase_char_id', 'freebase_a_id']
characters = pd.read_csv(CHARACTERS_FILE, sep='\t', names=character_columns, index_col=False)

characters['m_release_date'] = pd.to_datetime(characters['m_release_date'], format='%Y-%m-%d', errors='coerce')

print(characters.shape)
characters

(450669, 13)


,wiki_movie_id,freebase_movie_id,m_release_date,name,a_dob,a_gender,a_height,a_ethnicity_freebase_id,a_name,a_age_at_release,freebase_char/a_map,freebase_char_id,freebase_a_id
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
450664,913762,/m/03pcrp,1992-05-21,Elensh,1970-05,F,NaN,NaN,Dorothy Elias-Fahn,NaN,/m/0kr406c,/m/0kr406h,/m/0b_vcv
450665,913762,/m/03pcrp,1992-05-21,Hibiki,1965-04-12,M,NaN,NaN,Jonathan Fahn,27.0,/m/0kr405_,/m/0kr4090,/m/0bx7_j
450666,28308153,/m/0cp05t9,1957-01-01,NaN,1941-11-18,M,1.730,/m/02w7gg,David Hemmings,15.0,/m/0g8ngmc,NaN,/m/022g44
450667,28308153,/m/0cp05t9,1957-01-01,NaN,NaN,NaN,NaN,NaN,Roberta Paterson,NaN,/m/0g8ngmj,NaN,/m/0g8ngmm


In [25]:
characters = characters.join(movies[["freebase_movie_id", "countries"]].set_index("freebase_movie_id"), on="freebase_movie_id")
characters

,wiki_movie_id,freebase_movie_id,m_release_date,name,a_dob,a_gender,a_height,a_ethnicity_freebase_id,a_name,a_age_at_release,freebase_char/a_map,freebase_char_id,freebase_a_id,countries
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7,[United States of America]
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4,[United States of America]
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l,[United States of America]
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc,[United States of America]
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg,[United States of America]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450664,913762,/m/03pcrp,1992-05-21,Elensh,1970-05,F,NaN,NaN,Dorothy Elias-Fahn,NaN,/m/0kr406c,/m/0kr406h,/m/0b_vcv,[Japan]
450665,913762,/m/03pcrp,1992-05-21,Hibiki,1965-04-12,M,NaN,NaN,Jonathan Fahn,27.0,/m/0kr405_,/m/0kr4090,/m/0bx7_j,[Japan]
450666,28308153,/m/0cp05t9,1957-01-01,NaN,1941-11-18,M,1.730,/m/02w7gg,David Hemmings,15.0,/m/0g8ngmc,NaN,/m/022g44,[United Kingdom]
450667,28308153,/m/0cp05t9,1957-01-01,NaN,NaN,NaN,NaN,NaN,Roberta Paterson,NaN,/m/0g8ngmj,NaN,/m/0g8ngmm,[United Kingdom]


In [26]:
target_countries = ['Serbia']
filter_with_countries(characters, target_countries, mode='only')

,wiki_movie_id,freebase_movie_id,m_release_date,name,a_dob,a_gender,a_height,a_ethnicity_freebase_id,a_name,a_age_at_release,freebase_char/a_map,freebase_char_id,freebase_a_id,countries
60332,30320975.0,/m/0g5qxtx,2010-10-06,Mina,1981-11-17,F,1.78,/m/04kbvpz,Bojana Novakovic,28.0,/m/0gw3nfy,/m/0h37y2c,/m/02v_6qy,[Serbia]
124895,14882264.0,/m/03h0b44,2008-01-30,Arsa 'Kralj carlstona',1978-12-01,M,1.93,/m/02ctzb,Stefan Kapicic,29.0,/m/04tnnf1,/m/0hngk1g,/m/04tnnf4,[Serbia]
435377,34470730.0,/m/0h34t39,2012-03-13,Sef tuzilastva,1946-07-27,M,1.91,/m/09vmyh,Rade Serbedzija,65.0,/m/0h34t2_,/m/0h34t32,/m/034jgf,[Serbia]


In [27]:
target_countries = ['Serbia']
filter_with_countries(characters, target_countries, mode='any')

,wiki_movie_id,freebase_movie_id,m_release_date,name,a_dob,a_gender,a_height,a_ethnicity_freebase_id,a_name,a_age_at_release,freebase_char/a_map,freebase_char_id,freebase_a_id,countries
60332,30320975.0,/m/0g5qxtx,2010-10-06,Mina,1981-11-17,F,1.780,/m/04kbvpz,Bojana Novakovic,28.0,/m/0gw3nfy,/m/0h37y2c,/m/02v_6qy,[Serbia]
63757,13598172.0,/m/03cbh45,1982-01-01,Doktor Grujic,1946-07-27,M,1.910,/m/09vmyh,Rade Serbedzija,35.0,/m/040l3fw,/m/0h34zkm,/m/034jgf,"[Serbia, Yugoslavia]"
124895,14882264.0,/m/03h0b44,2008-01-30,Arsa 'Kralj carlstona',1978-12-01,M,1.930,/m/02ctzb,Stefan Kapicic,29.0,/m/04tnnf1,/m/0hngk1g,/m/04tnnf4,[Serbia]
298431,16133349.0,/m/03wb067,2006-01-01,Brewster,1943-03-31,M,1.830,/m/01qhm_,Christopher Walken,62.0,/m/04dfw24,/m/0gy8p22,/m/016fjj,"[Italy, Serbia, United Kingdom]"
298432,16133349.0,/m/03wb067,2006-01-01,Tommaso Moreno,1979-12-29,M,1.780,/m/09k5jvk,Diego Luna,26.0,/m/04dfw29,/m/0h34tg5,/m/037kqv,"[Italy, Serbia, United Kingdom]"
298433,16133349.0,/m/03wb067,2006-01-01,Lea Padovani,1976-01-02,F,1.670,/m/03ttfc,Paz Vega,29.0,/m/04dfw2g,/m/05kc0k7,/m/04mdbq,"[Italy, Serbia, United Kingdom]"
309854,28135570.0,/m/0cmcv5z,2010-07-23,Lucija,1955-09-07,F,1.676,/m/041rx,Mira Furlan,54.0,/m/0cmxv87,/m/0j4_lhq,/m/0prs1,"[Belgium, Bosnia and Herzegovina, Slovenia, Fr..."
435377,34470730.0,/m/0h34t39,2012-03-13,Sef tuzilastva,1946-07-27,M,1.910,/m/09vmyh,Rade Serbedzija,65.0,/m/0h34t2_,/m/0h34t32,/m/034jgf,[Serbia]
443170,27128043.0,/m/0bwhwdx,2006-09-28,Marina,1981-11-17,F,1.780,/m/04kbvpz,Bojana Novakovic,24.0,/m/0g9hd4k,/m/0h37y30,/m/02v_6qy,"[Switzerland, Spain, Serbia, Monaco]"
